### Documentation and Notes


**Data Analytics Case Study - Task 1 - Group 11**

In the following, we will describe the development of a dataset generator. We decided on creating a dataset for social media account information containing 9 different features: in addition to an ID to keep track, we create the following information: Username, Age, Country, CreationDate, Description, FollowerCount, PostCount, and Language. We tried to incorporate several numerical features with different distributions and characteristics to be analyzed, as well as some textual data for text analysis.


**Parameters:**

There are several parameters that can be set in the creation in order to design the data. While the parameters can be set in the following cell of the Notebook, they namely include:

- Output Count: the number of output entries to generate
- Noise: a factor between 0 and 1 describing how noisy the created data should be 
- Bots: the percentage (between 0 and 1) of output results that are created as bot-accounts (creation-probability later follows uniform distribution)
- Companies: the percentage (between 0 and 1) of output results that are created as company-accounts (creation-probability later follows uniform distribution)
- NA probability: the percentage (between 0 and 1) of NA-entries in the output results (creation-probability later follows uniform distribution)

note: the creation of a dataset with a reasonable number of entries may take a while. Reason for that is mainly the generation of the text description using an AI-generator. It is timely to replicate, which is why we added an exemplary dataset in the submission as well. However, Chat-GPT text generator does not allow to set a seed for creation for replication.


**Setup:**

In addition to the variable parameters to be set prior to the execution, there are several setup variables for the creation of the final output. These include a list of topics that is further used to create prompts for the text description generator. There is a list of topics for each type of account, i.e., bot, company or 'normal' accounts. We tried to collect topics that are typical for the types, respectively. In addition, we listed multiple languages and countries to use in the creation.

- Set possible description topics.
- Set bot description topics.
- Set company description topics.
- Set possible countries and languages.


**Process and Features:**

In the following, the general procedure of our dataset generator is briefly described with exemplary values to facilitate the understanding of the code. In addition, more information about the created features is given including for example correlations and distributions of the data.

Please set the input parameters in the first cell, the setup lists in the second one can be kept without changing. Then, execute the following cells.

For each entry:
1. **First:** Classification as Bot or Not (e.g. Probability: 0.05)
2. **Second:** Private company classification (e.g. Probability: 0.1 company):

// create Age (follows normal distribution)
- If company:
  - Age = NA
- Else:
  - Age = Random (NA possible).

// create CreationDate
- If Bot:
  - CreationDate set as more recent.
- If not Bot:
  - CreationDate set as completely random.

// create Language and Country
- Languages set as random (NA possible).
- Location and Language can correlate (NA possible).

// create PostCount (correlated with CreationDate)
- If Bot:
  - Post Count high.
- If not Bot:
  - Random post count.
  
// create FollowerCount (correlated with CreationDate)
- If Bot:
  - Follower count low.
- If not Bot:
  - Follower count random but minimal, correlated with creation date.
  

// create Description and Username (using Chat-GPT, as the creation of the descriptions is based on the pre-specified lists of topics, the resulting data could potentially be clustered by topic)
- If Bot:
  - Description based on bot topics.
  - Name generated with description topics.
- If not Bot:
  - If company:
    - Use company topics.
    - Name generated with description topics.
  - If not company:
    - Description based on 'normal' topics.
    - Name generated with description topics.


## Generator

### Inputs:

In [1]:
# Parameters to be set
OutputCount = 500 # Number of output Results
Noise = 1 # Factor between 0 and 1
Bots = 0.05 # Percentage of Bots
Company = 0.1 # Percentage of Company accounts
NA_Prob = 0.05 # Percentage of NAs

In [2]:
# Setup
# Description Topics
BotDesc = ["Ukraine war","Vote","Republican","Follow me","Trump","Democracy","Bad","Hate","Dumb","Covid"]
CompanyDesc = ["Profit","Business","Fiscal year","2023","Expansion","Success","Job offer","Event","Margin","Good vibes"]
NormalDesc = ["Animals","Sport","Gaming","2023","Travel","Fun",":)","Event","Job","taking Images","Politics","Republicans"]

# Possible Countries and Languages
Countries = ["Germany","France","USA","UK","China","Japan","Russia","Italy","Netherlands","Switzerland"]
Languages = ["GER","FRE","ENG","ENG","MAN","JAP","RUS","ITA","NED","GER"]

### Code

In [3]:
# All these installations should be done.
# !pip install pandas
# !pip install numpy
# !pip install random
# !pip install faker
# !pip install torchvision
# !pip install warnings
# Imports
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import random
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
import faker
import warnings
warnings.filterwarnings("ignore")

C:\Users\domin\anaconda3\envs\DA1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# create empty panda DF
data = {
    'ID': [],
    'Username': [],
    'Age': [],
    'Country': [],
    'CreationDate': [],
    'Description': [],
    'FollowerCount': [],
    'PostCount': [],
    'Language': []
}

df = pd.DataFrame(data)

row = {'ID':'','Username': '','Age': '','Country': '','CreationDate': '','Description': '','FollowerCount': '','PostCount': '','Language': ''}

In [5]:
# create dates
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y %I:%M %p', prop)

In [6]:
# instantiate the AI-model for text generation

rng = np.random.default_rng(1)
fake = faker.Faker()
model_name = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model_name = "meta-llama/Llama-2-7b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# function-creation for the features following the characteristics as described in the introductory documentation

def decide_account_type(bot, company, Topicslist):
    bot = False
    company = False
    Topicslist = []

    def append_topics(desc_list, count):
        for _ in range(count):
            n = random.randint(0, len(desc_list) - 1)
            Topicslist.append(n)

    # Setting whether the account is a bot account
    if rng.uniform() < Bots:
        bot = True
        append_topics(BotDesc, 3)

    # Setting whether the account is a company account
    elif rng.uniform() < Company:
        company = True
        append_topics(CompanyDesc, 3)

    # Setting the topiclist-IDs, when the account is a normal account
    else:
        append_topics(NormalDesc, 3)

    return bot, company, Topicslist


# Setting the age
def set_age(company):
    if company or rng.uniform() < NA_Prob:
        return ""  # or return None

    age = np.random.normal(loc=30, scale=25, size=None)
    while age < 16:
        age = np.random.normal(loc=30, scale=25, size=None)

    return round(age)


# Setting the creation date
def set_creation_date(bot):
    if bot:
        return random_date("1/1/2020 1:30 PM", "1/24/2024 11:59 PM", random.random())
    else:
        return random_date("1/1/2008 1:30 PM", "1/24/2024 11:59 PM", random.random())


# setting the country and language
def set_country_language():
    rand_country = round(random.randrange(0, 9))
    if rng.uniform() < NA_Prob:
        country = ""
    else:
        country = Countries[rand_country]

    if rng.uniform() < 0.4:
        rand_language = round(random.randrange(0, 9))
        language = Languages[rand_language]
    else:
        language = Languages[rand_country]
    return country, language


# Calculating the age of the account
def calculate_account_age(creation_date):
    account_date_difference = datetime.strptime(
        "1/24/2024 11:59 PM", "%m/%d/%Y %I:%M %p"
    ) - datetime.strptime(creation_date, "%m/%d/%Y %I:%M %p")
    return abs(account_date_difference.days)


# Calculating the FollowerCount & PostCount based on the account age
def calculate_followers_posts(bot, account_age):
    if bot:
        f_multiplier = 0.5
        p_multiplier = random.randrange(10, 30)
        f_loc = 0.1*Noise
        f_scale = 0.2*Noise
        p_loc = 0.1*Noise
        p_scale = 0.5*Noise
    else:
        f_multiplier = 1*Noise
        p_multiplier = 0.1*Noise
        f_loc = 0.4*Noise
        f_scale = 2*Noise
        p_loc = 0.05*Noise
        p_scale = 0.1*Noise

    follower_count = (
        np.random.normal(loc=f_loc, scale=f_scale, size=None)
        * account_age
        * f_multiplier
    )

    post_count = (
        np.random.normal(loc=p_loc, scale=p_scale, size=None)
        * account_age
        * p_multiplier
    )

    return abs(round(follower_count)), abs(round(post_count))


# Define the random username of the user
def generate_username():
    if random.uniform(0, 1) < 0.5:
        return fake.user_name()
    else:
        if random.uniform(0, 1) < 0.5:
            return (
                str(random.randrange(1950, 2005)) + "_" + fake.user_name()
            )
        else:
            return fake.user_name() + "_" + str(random.randrange(1950, 2005))

        
# generate the text description
def generate_description(
    bot,
    company,
    Topicslist,
    username,
    BotDesc,
    CompanyDesc,
    NormalDesc,
    model,
    tokenizer,
):
    def construct_prompt(entity_type, desc_list):
        topics = ", ".join(str(desc_list[topic]) for topic in Topicslist[:3])
        return f"{username}, topics: {topics}, short profile description:"

    def clean_generated_text(text, prompt):
        text = text.replace(prompt, "").strip()  # Remove prompt if present in the result
        for char in ["\n", ")", '"']:
            text = text.replace(char, "")
        return text

    if bot:
        prompt = construct_prompt("bot", BotDesc)
    elif company:
        prompt = construct_prompt("company", CompanyDesc)
    else:
        prompt = construct_prompt("user", NormalDesc)

    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    output = model.generate(input_ids, max_new_tokens=15, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    result = clean_generated_text(generated_text, prompt)

    return result

In [8]:
# Main loop executing all creations and generating data for the final dataset
for i in range(1, OutputCount + 1):
    row = {
        "ID": "",
        "Username": "",
        "Age": "",
        "Country": "",
        "CreationDate": "",
        "Description": "",
        "FollowerCount": "",
        "PostCount": "",
        "Language": "",
    }
    row["ID"] = i
    bot = False
    company = False
    Topicslist = []
    
    bot, company, Topicslist = decide_account_type(bot, company, Topicslist) # Helpers

    row["Age"] = set_age(company)
    
    creation_date = set_creation_date(bot)
    row["CreationDate"] = creation_date
    
    row["Country"], row["Language"] = set_country_language()
    
    account_age = calculate_account_age(creation_date)
    
    row["FollowerCount"], row["PostCount"] = calculate_followers_posts(bot, account_age)
    
    username = generate_username()
    row["Username"] = username
    
    row["Description"] = generate_description(
        bot,
        company,
        Topicslist,
        username,
        BotDesc,
        CompanyDesc,
        NormalDesc,
        model,
        tokenizer,
    )

    print(f"Finished: row {row['ID']} of {OutputCount}")
    print(row)
    
    df.loc[len(df)] = row

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Finished: row 1 of 500
{'ID': 1, 'Username': 'mark51', 'Age': 39, 'Country': 'Germany', 'CreationDate': '02/29/2016 06:00 PM', 'Description': 'I am a gamer, and I am a gamer for a reason', 'FollowerCount': 2673, 'PostCount': 24, 'Language': 'GER'}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finished: row 2 of 500
{'ID': 2, 'Username': 'lucerocarol_1981', 'Age': 27, 'Country': 'China', 'CreationDate': '12/13/2023 07:50 AM', 'Description': 'The first of the two species of the genus _Lucerocarol', 'FollowerCount': 161, 'PostCount': 1, 'Language': 'GER'}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Finished: row 497 of 500
{'ID': 497, 'Username': 'baileysandra', 'Age': 41, 'Country': 'China', 'CreationDate': '06/05/2022 02:26 AM', 'Description': 'I am a full time student at the University of Toronto. I am a', 'FollowerCount': 1561, 'PostCount': 3, 'Language': 'ENG'}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finished: row 498 of 500
{'ID': 498, 'Username': 'floresbeth_1975', 'Age': 84, 'Country': 'China', 'CreationDate': '07/21/2015 09:20 AM', 'Description': 'I am a young, single, and very ambitious woman.', 'FollowerCount': 5259, 'PostCount': 58, 'Language': 'MAN'}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Finished: row 499 of 500
{'ID': 499, 'Username': 'henglish', 'Age': '', 'Country': 'UK', 'CreationDate': '03/08/2015 01:56 AM', 'Description': 'The first Chinese-language newspaper in the United States, the People', 'FollowerCount': 5053, 'PostCount': 58, 'Language': 'ENG'}
Finished: row 500 of 500
{'ID': 500, 'Username': 'pspencer_1969', 'Age': 44, 'Country': 'Japan', 'CreationDate': '12/20/2022 11:37 AM', 'Description': 'The first ever World Cup of Golf was held in the United States in 1969', 'FollowerCount': 228, 'PostCount': 1, 'Language': 'JAP'}


### Output Documentation

- ID: ID of the entry, i.e., to keep track of the accounts in the dataset
- Username: username of the user, i.e., the account in focus
- Age: age of the user, may contain NAs
- Country: country of the user
- CreationDate: date and time when the account was created
- Description: textual description that is displayed on the account
- FollowerCount: number of users that follow the account in focus
- PostCount: number of posts that were uploaded on the account in focus
- Language: language of the user

In [9]:
df.to_csv('data_example.csv', index=False)

# display(df['Description'])